In [9]:
import numpy as np
import rasterio
import joblib
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt

# Load the TIF imagery and binary mask
with rasterio.open('/home/jovyan/Desktop/shivanshi thesis/Tushar/Internship/30_bands_final.tif') as src:
    imagery = src.read(1).astype(np.float32)

with rasterio.open('/home/jovyan/Desktop/shivanshi thesis/Tushar/Internship/Mask.tif') as src:
    mask = src.read(1).astype(np.uint8)

# Create a valid mask excluding NaN values
valid_mask = ~np.isnan(imagery)

# Flatten the imagery and mask arrays to 1D
X = imagery[valid_mask].reshape(-1, 1)
y = mask[valid_mask].reshape(-1)

# Define class names and their corresponding indices
class_names = ['non-peat', 'peat']
class_indices = [0, 1]

# Keep only the valid classes '0' (non-peat) and '1' (peat)
valid_classes_mask = np.isin(y, class_indices)
X = X[valid_classes_mask]
y = y[valid_classes_mask]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
# Perform grid search to find the best hyperparameters for the SVM model
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
}

svm_model = SVC(class_weight='balanced', probability=True, random_state=42)
grid_search = GridSearchCV(svm_model, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best SVM model from grid search
best_svm_model = grid_search.best_estimator_

# Save the best SVM model as .sav file
joblib.dump(best_svm_model, '/home/jovyan/Desktop/shivanshi thesis/Tushar/Internship/svm_model.sav')




In [ ]:
# Calculate prediction probabilities for each class
prediction_probabilities = best_svm_model.predict_proba(X_test)

# Categorize probabilities into confidence intervals
conf_intervals = [0, 0.45, 0.60, 0.90, 0.95, 1.0]
interval_labels = ['Very Low Confidence', 'Low Confidence', 'Intermediate Confidence', 'High Confidence', 'Very High Confidence']

# Create empty arrays for each confidence interval
probabilities_peat_intervals = np.zeros_like(prediction_probabilities[:, 1], dtype=np.uint8)
probabilities_no_peat_intervals = np.zeros_like(prediction_probabilities[:, 0], dtype=np.uint8)

# Categorize probabilities for peat class
for i in range(len(conf_intervals) - 1):
    mask = (conf_intervals[i] <= prediction_probabilities[:, 1]) & (prediction_probabilities[:, 1] < conf_intervals[i + 1])
    probabilities_peat_intervals[mask] = i + 1

# Categorize probabilities for non-peat class
for i in range(len(conf_intervals) - 1):
    mask = (conf_intervals[i] <= prediction_probabilities[:, 0]) & (prediction_probabilities[:, 0] < conf_intervals[i + 1])
    probabilities_no_peat_intervals[mask] = i + 1

# Calculate percentages for each confidence interval
percentages_peat = [np.sum(probabilities_peat_intervals == i) / len(probabilities_peat_intervals) for i in range(1, len(interval_labels) + 1)]
percentages_no_peat = [np.sum(probabilities_no_peat_intervals == i) / len(probabilities_no_peat_intervals) for i in range(1, len(interval_labels) + 1)]

# Plot the bar graph
bar_width = 0.35
index = np.arange(len(interval_labels))

plt.bar(index, percentages_peat, bar_width, label='Peat', color='green')
plt.bar(index + bar_width, percentages_no_peat, bar_width, label='Non-Peat', color='blue')

plt.xlabel('Confidence Intervals')
plt.ylabel('Percentage')
plt.title('Percentage of Confidence Intervals for Peat and Non-Peat')
plt.xticks(index + bar_width / 2, interval_labels, rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
# Calculate confusion matrix, precision, recall, F1 score, and accuracy
y_pred = best_svm_model.predict(X_test)
confusion_mat = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=1)
accuracy = accuracy_score(y_test, y_pred)

print('Confusion Matrix:')
print(confusion_mat)
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'Accuracy: {accuracy}')



In [ ]:
# Save the prediction probabilities of peat and non-peat as separate TIF files
probabilities_peat = np.zeros_like(imagery, dtype=np.float32)
probabilities_no_peat = np.zeros_like(imagery, dtype=np.float32)
probabilities_peat[valid_mask] = prediction_probabilities[:, 1]
probabilities_no_peat[valid_mask] = prediction_probabilities[:, 0]

with rasterio.open('/home/jovyan/Desktop/shivanshi thesis/Tushar/Internship/probability_peat.tif', 'w', driver='GTiff', height=imagery.shape[0], width=imagery.shape[1], count=1, dtype=np.float32, crs=src.crs, transform=src.transform) as dst:
    dst.write(probabilities_peat, 1)

with rasterio.open('/home/jovyan/Desktop/shivanshi thesis/Tushar/Internship/probability_no_peat.tif', 'w', driver='GTiff', height=imagery.shape[0], width=imagery.shape[1], count=1, dtype=np.float32, crs=src.crs, transform=src.transform) as dst:
    dst.write(probabilities_no_peat, 1)

# Save the classified image as a new TIF file
classified_output = np.zeros_like(imagery, dtype=np.uint8)
classified_output[valid_mask] = y_pred
with rasterio.open('/home/jovyan/Desktop/shivanshi thesis/Tushar/Internship/classified_image.tif', 'w', driver='GTiff', height=imagery.shape[0], width=imagery.shape[1], count=1, dtype=np.uint8, crs=src.crs, transform=src.transform) as dst:
    dst.write(classified_output, 1)